In [1]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io, transform
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from PIL import Image

import PIL

# Main
This notebook help to determine some hyperparameters for acquiring the best test result and generate .csv for upload

In [2]:
class imagDataset_test(Dataset):
    # data set get image data 
    """
    Data set to import imag
    """

    def __init__(self,  root_dir, transform = None):
        self.root_dir   = root_dir
        self.transform  = transform
        self.file_name = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
        

    def __len__(self):
        return len(self.file_name)
    
    
    def __getitem__(self, item):
        if( torch.is_tensor(item)):
            item = item.tolist()

        img_name = os.path.join( self.root_dir, self.file_name[item])
        #image = io.imread(img_name)
        image = Image.open(img_name)
        image = image.convert('RGB')
        
        if(self.transform):
            image = self.transform(image)
            
        sample = {'image':image, 'label':item}
        
        return sample

    def get_file_name(self):
        return self.file_name



def get_test_data(root,fig_size, batch_size):
    # ensemble data loader
    # use three data pre-processing methods 
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

    trans = transforms.Compose([
                transforms.Resize((300,300)),
                transforms.CenterCrop(fig_size),
                transforms.ToTensor(),
                normalize,
            ])
    trans2 = transforms.Compose([
                transforms.Resize((fig_size,fig_size)),
                transforms.ToTensor(),
                normalize])
    trans3=transforms.Compose([               
                transforms.RandomRotation(degrees=90),
                transforms.RandomResizedCrop(fig_size,scale=(0.05, 1.0), ratio=(0.75, 1.3333333333333333)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize])
    test_data = imagDataset_test(root, transform=trans)
    test_data2= imagDataset_test(root, transform=trans2)
    test_data3= imagDataset_test(root, transform=trans3)
    test_name = test_data.get_file_name()
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)
    test_dataloader2 = DataLoader(test_data2, batch_size=batch_size, shuffle=False, num_workers=0)
    test_dataloader3 = DataLoader(test_data3, batch_size=batch_size, shuffle=False, num_workers=0)
    print("num of data",len(test_data))
    return [test_dataloader, test_dataloader2,test_dataloader3,test_name]



class chain_model(nn.Module): # define the model here for load model 
    def __init__(self, pre_trained, num_classes):
        super(chain_model, self).__init__()
        self.pre_trained = pre_trained # pre trained model 
        
        for param in self.pre_trained.parameters():
            param.requires_grad = False # don't do gradient descent for bottom part 
                    
        in_feature = self.pre_trained.fc.in_features
        
        feature_1 = 2000
        feature_2 = 1000
        feature_3 = num_classes
        
        self.pre_trained.fc = nn.Linear(in_feature, feature_1)
        self.rl1 = nn.LeakyReLU()
        self.fc1 = nn.Linear(feature_1, feature_2)
        self.rl2 = nn.LeakyReLU()
        self.fc2 = nn.Linear(feature_2, feature_3)
    
        
    
    def forward(self, x):
        x = self.pre_trained(x)
        x = self.rl1(x)
        x = self.fc1(x)
        x = self.rl2(x)
        x = self.fc2(x)
        
        return x 
        
        
def load_model(path):
    if( not os.path.isfile(path)):
        print(" model doesn't exist")
        return None
    
    check_point = torch.load(path)

    model      = check_point['model']
    print("load model")

    return model

In [11]:
def print_predict(test_data, test_data2,test_data3,test_file, model_ensemble,weight, device):
    """
    test_data: data loader
    test_data2:
    test_data3:
    data loader with different data pre-processing methods 
    
    test_file: list of file name 
    model ensemble: list of model to do ensemble 
    weight: weight for each ensemble
    
    """
    soft_max =nn.Softmax(dim=1)
    for model in model_ensemble:
        model.eval() 
    with torch.no_grad():
        name_list = []
        class_list = []
        it2=iter(test_data2)
        it3=iter(test_data3)
        for i_batch, sample_batched in enumerate(test_data):
            data2=next(it2)
            data3=next(it3)
            x = sample_batched['image'].to(device=device, dtype=torch.float32)
            x2 =data2['image'].to(device=device, dtype=torch.float)
            x3 =data3['image'].to(device=device, dtype=torch.float)
            ind = sample_batched['label'].to(device=device, dtype=torch.long) 
            # read data by different data loader
            
            score_1 = weight[0] * soft_max(model_ensemble[0](x))[:,0:251]
            score_2 = weight[0] * soft_max(model_ensemble[0](x2))[:,0:251]
            score_3 = weight[0] * soft_max(model_ensemble[0](x3))[:,0:251]
            scores  = score_1 * 0.75 + score_2 * 0.425 + score_3 * 0.25 # weighted average for each output 
            
            for i in range(1,len(model_ensemble)): # weighted average for each model
                score_1 = weight[i] * soft_max(model_ensemble[i](x))[:,0:251]
                score_2 = weight[i] * soft_max(model_ensemble[i](x2))[:,0:251]
                score_3 = weight[i] * soft_max(model_ensemble[i](x3))[:,0:251]
                scores  += score_1 * 0.75 + score_2 * 0.425 + score_3 * 0.25 # weighted average of each output 
            # Ensemble method for x1,x2,x3 input. Ensemble method for models 
            
            
            _,preds=scores.topk(3,1) 
 
            name_list = name_list + [test_file[i] for i in ind]
            class_list = class_list + preds.tolist()

            ans=np.hstack((np.asarray(name_list).reshape((len(name_list),1)),np.asarray(class_list)))
            # get top three prediction 
            
            if( i_batch %50 == 0):
                print("ibatch = ",i_batch, len(class_list), len(name_list))
        df=pd.DataFrame(ans)
        return df

In [4]:
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)


using device: cuda


In [6]:

fig_size = 224
batch_size = 16
path_test_data = '../input/project-ifood/test_set/test_set' # path to the test set 
test1,test2,test3, filename =  get_test_data(path_test_data,fig_size, batch_size) # read model 


/kaggle/working
num of data 28377


In [7]:
path_list = ["../input/model-ensemble/res101_16", "../input/model-ensemble/res152",
              "../input/res152net3fc/wideres50fc3", "../input/res152net3fc/res152fc3",
             '../input/model-ensemble/resnext_13_model_only/model_only_13' ]
# list of path of each model 
weight =  [3,3,3,2,2]
# weight of each model 

model_list = []
for path_model in path_list:
    model_list.append(load_model(path_model)[0] )
# read the model list

../input/model-ensemble/res101_16
../input/model-ensemble/res101_16
load model
../input/model-ensemble/res152
../input/model-ensemble/res152
load model
../input/res152net3fc/wideres50fc3
../input/res152net3fc/wideres50fc3
load model
../input/res152net3fc/res152fc3
../input/res152net3fc/res152fc3
load model
../input/model-ensemble/resnext_13_model_only/model_only_13
../input/model-ensemble/resnext_13_model_only/model_only_13


In [12]:
path_save = '../working/ensemble_4.csv'  # save the prediction to this path 
df = print_predict(test1,test2,test3,filename, model_list, weight,device) # get prediction 
df.columns = ['img_name','label1', 'label2', 'label3' ]
df['label']=df.apply(lambda x:' '.join(str(x) for x in x[1:]),axis=1)
df=df.loc[:,['img_name','label']]
df.to_csv(path_save, index=False) # save 

ibatch =  0 16 16
ibatch =  50 816 816
ibatch =  100 1616 1616
ibatch =  150 2416 2416
ibatch =  200 3216 3216
ibatch =  250 4016 4016
ibatch =  300 4816 4816
ibatch =  350 5616 5616
ibatch =  400 6416 6416
ibatch =  450 7216 7216
ibatch =  500 8016 8016
ibatch =  550 8816 8816
ibatch =  600 9616 9616
ibatch =  650 10416 10416
ibatch =  700 11216 11216
ibatch =  750 12016 12016
ibatch =  800 12816 12816
ibatch =  850 13616 13616
ibatch =  900 14416 14416
ibatch =  950 15216 15216
ibatch =  1000 16016 16016
ibatch =  1050 16816 16816
ibatch =  1100 17616 17616
ibatch =  1150 18416 18416
ibatch =  1200 19216 19216
ibatch =  1250 20016 20016
ibatch =  1300 20816 20816
ibatch =  1350 21616 21616
ibatch =  1400 22416 22416
ibatch =  1450 23216 23216
ibatch =  1500 24016 24016
ibatch =  1550 24816 24816
ibatch =  1600 25616 25616
ibatch =  1650 26416 26416
ibatch =  1700 27216 27216
ibatch =  1750 28016 28016
